# MATLAB to Python Optimization Conversion

This notebook converts MATLAB optimization code (excluding JAYA) to Python using optimization libraries and the pandapower 33-bus test case.

## Import Required Libraries
Import necessary Python libraries such as pandapower, scipy.optimize, numpy, and matplotlib.

In [ ]:
# Import necessary libraries
import pandapower as pp
import pandapower.networks as nw
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt

# Optional: Configure plotting backend
# %matplotlib inline

## Load Pandapower 33-Bus Test Case
Load the 33-bus test case from pandapower and display the network structure.

In [ ]:
# Load the IEEE 33-bus test case from pandapower.networks
# Note: Pandapower might not have a direct 33-bus case named exactly like this.
# We might need to use a similar one like case14, case30, or load it from a file if available.
# Using case_simbench as an example, replace with the correct 33-bus case loading if needed.
# For demonstration, let's load a standard case like case14.
# If you have a specific file for the 33-bus system (e.g., in MATPOWER format),
# you might need a converter or load it differently.

try:
    # Attempt to load a common benchmark case if 33-bus isn't directly available
    net = nw.case14()
    # Or try loading by name if available in your pandapower version
    # net = pp.from_json("path/to/your/33bus.json") or pp.from_excel("path/to/your/33bus.xlsx")
    # net = nw.case33bw() # Check if this exists in your version
    print("Successfully loaded a test case.")
except Exception as e:
    print(f"Error loading network: {e}")
    print("Please ensure the specified test case is available or provide the correct path/name.")
    net = None

# Display basic network information
if net:
    print("Network Summary:")
    print(net)

    # Optionally, plot the network
    # import pandapower.plotting as plot
    # plot.simple_plot(net, show_plot=True)

## Define Optimization Problem
Define the optimization problem using scipy.optimize or other Python optimization libraries. Include constraints and objective functions based on the MATLAB code.

In [ ]:
# Define the objective function based on the MATLAB code
# Example: Minimize losses or generation cost
def objective_function(x, net):
    # 'x' represents the optimization variables (e.g., generator setpoints, tap positions)
    # Update the network based on 'x'
    # net.gen['p_mw'][gen_indices] = x[0:num_gens]
    # net.trafo['tap_pos'][trafo_indices] = x[num_gens:]

    # Run power flow
    try:
        pp.runpp(net)
    except pp.LoadflowNotConverged:
        return np.inf # Return a large value if power flow doesn't converge

    # Calculate the objective (e.g., total active power loss)
    loss = net.res_loss.p_mw.sum()
    return loss

# Define constraints based on the MATLAB code
# Example: Generator limits, voltage limits, line loading limits
cons = []

# Example: Voltage limits (lower and upper bounds)
# vm_pu_min = 0.95
# vm_pu_max = 1.05
# cons.append({'type': 'ineq', 'fun': lambda x: net.res_bus.vm_pu - vm_pu_min}) # vm_pu >= vm_pu_min
# cons.append({'type': 'ineq', 'fun': lambda x: vm_pu_max - net.res_bus.vm_pu}) # vm_pu <= vm_pu_max

# Example: Generator active power limits
# p_min = net.gen['min_p_mw']
# p_max = net.gen['max_p_mw']
# Define bounds for optimization variables 'x'
# bounds = [(p_min_i, p_max_i) for p_min_i, p_max_i in zip(p_min, p_max)] + [...] # Add bounds for other variables like tap positions

# Placeholder for bounds and constraints - these need to be defined based on the specific problem
bounds = None # Define bounds based on variable limits (e.g., generator P, Q, tap pos)
# constraints = ({'type': 'eq', 'fun': constraint_eq}, # Equality constraints
#                {'type': 'ineq', 'fun': constraint_ineq}) # Inequality constraints

print("Objective function and constraints structure defined (placeholders).")
# Note: The actual implementation requires mapping 'x' to network parameters
# and defining specific constraint functions that evaluate feasibility after a power flow.

## Solve Optimization Problem
Use the optimization library to solve the defined problem and extract the results.

In [ ]:
# Initial guess for the optimization variables
# x0 = np.concatenate([net.gen['p_mw'].values, net.trafo['tap_pos'].values]) # Example initial guess
x0 = None # Define an appropriate initial guess based on the variables

# Choose an optimization algorithm (e.g., 'SLSQP', 'trust-constr')
solver_method = 'SLSQP'

# Run the optimization
if net and x0 is not None and bounds is not None:
    print(f"Starting optimization using {solver_method}...")
    result = minimize(objective_function,
                      x0,
                      args=(net,),
                      method=solver_method,
                      bounds=bounds,
                      constraints=cons, # Use 'cons' defined earlier
                      options={'disp': True}) # Display solver output

    # Check if the optimization was successful
    if result.success:
        print("Optimization successful!")
        optimized_x = result.x
        optimal_objective_value = result.fun
        print("Optimal Objective Value:", optimal_objective_value)
        print("Optimal Variables (x):", optimized_x)

        # Update the network with the optimal values for final analysis
        # (Similar logic as in the objective function)
        # net.gen['p_mw'][gen_indices] = optimized_x[0:num_gens]
        # net.trafo['tap_pos'][trafo_indices] = optimized_x[num_gens:]
        # pp.runpp(net) # Run final power flow

    else:
        print("Optimization failed.")
        print("Failure message:", result.message)
        optimized_x = None
else:
    print("Optimization prerequisites not met (network not loaded, x0/bounds not defined).")
    result = None
    optimized_x = None

## Analyze and Visualize Results
Analyze the optimization results and visualize them using matplotlib or other visualization tools.

In [ ]:
# Analyze the results
if result and result.success and net:
    print("\n--- Analysis of Optimization Results ---")

    # Compare initial and final objective function values (if initial state was evaluated)
    # initial_loss = objective_function(x0, net) # Need to run power flow for x0 state first
    # print(f"Initial Objective Value (Loss): {initial_loss}") # Placeholder
    print(f"Optimal Objective Value (Loss): {optimal_objective_value}")

    # Display final network state results
    print("\nFinal Bus Voltages (p.u.):")
    print(net.res_bus.vm_pu)

    print("\nFinal Generator Power (MW):")
    print(net.res_gen.p_mw)

    print("\nFinal Line Loadings (%):")
    print(net.res_line.loading_percent)

    # Visualize the results
    plt.figure(figsize=(10, 6))

    # Example: Plot bus voltages
    plt.plot(net.res_bus.index, net.res_bus.vm_pu, marker='o', linestyle='-', label='Optimized Voltages')
    plt.axhline(1.05, color='r', linestyle='--', label='Upper Limit (1.05 pu)')
    plt.axhline(0.95, color='g', linestyle='--', label='Lower Limit (0.95 pu)')
    plt.xlabel("Bus Index")
    plt.ylabel("Voltage (p.u.)")
    plt.title("Bus Voltages after Optimization")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Example: Plot line loadings
    plt.figure(figsize=(10, 6))
    plt.plot(net.res_line.index, net.res_line.loading_percent, marker='x', linestyle='-', color='orange', label='Optimized Line Loading')
    plt.axhline(100, color='r', linestyle='--', label='Loading Limit (100%)')
    plt.xlabel("Line Index")
    plt.ylabel("Loading (%)")
    plt.title("Line Loading after Optimization")
    plt.legend()
    plt.grid(True)
    plt.show()

else:
    print("Analysis cannot be performed. Optimization did not succeed or network is not available.")